In [ ]:
pwd

u'/Users/auroraleport/Documents/LePort_git/Springboard_2016/Kick_Cap_2016'

In [ ]:
import pandas as pd
import glob

appended_data = []
for infile in glob.glob('/Users/auroraleport/Documents/Kickstarter_2016-07-15/*.csv'):
    print infile
    data = pd.read_csv(infile)
    data['filename'] = infile
    #print data
    appended_data.append(data) ## store dataframes in list
    print len(appended_data)
appended_data = pd.concat(appended_data, axis=0) ## see documentation for more info
#pd.DataFrame(appended_data)
#appended_data.to_csv('appedned.csv')
len(appended_data)
appended_data.head()

/Users/auroraleport/Documents/Kickstarter_2016-07-15/Kickstarter.csv
1
/Users/auroraleport/Documents/Kickstarter_2016-07-15/Kickstarter001.csv
2
/Users/auroraleport/Documents/Kickstarter_2016-07-15/Kickstarter002.csv
3
/Users/auroraleport/Documents/Kickstarter_2016-07-15/Kickstarter003.csv
4
/Users/auroraleport/Documents/Kickstarter_2016-07-15/Kickstarter004.csv
5
/Users/auroraleport/Documents/Kickstarter_2016-07-15/Kickstarter005.csv
6
/Users/auroraleport/Documents/Kickstarter_2016-07-15/Kickstarter006.csv
7
/Users/auroraleport/Documents/Kickstarter_2016-07-15/Kickstarter007.csv


In [ ]:
k_data = appended_data

In [ ]:
# extracting targeted values:
print k_data.columns.values
k_var = k_data[['category','name','blurb','currency','static_usd_rate','goal','backers_count','state','launched_at','state_changed_at']]
print len(k_var)
k_var.head()
k_var.reset_index(drop=True,inplace=True)

# Goals:
    # Want to convert blurb to count of words in blurb. (complete)
    # Need to extract category name from category url. (complete)
    # Need to convert launched_at time from Unix time to PST (complete)
    # Need to convert goal amount to USD currency units (complete)
    # Need to extract all states that are live from dataframe (complete)
    # Need to calculate time to state change (state_change - launched_at) (complete)

# Calculating time to state change

In [ ]:
import datetime
launch_date = []

for date in k_var['launched_at']:
    new_date = datetime.datetime.fromtimestamp(int(date)).strftime('%Y-%m-%d')
    launch_date.append(new_date)
launch_date = pd.to_datetime(launch_date)

change_date = []
for date in k_var['state_changed_at']:
    new_date2 = datetime.datetime.fromtimestamp(int(date)).strftime('%Y-%m-%d')
    change_date.append(new_date2)
change_date = pd.to_datetime(change_date)

#k_var['days_to_change'] = (pd.to_datetime(change_date['change_date']) - pd.to_datetime(launch_date['launch_date']))

In [ ]:
# print k_var.dtypes
k_var['launched_at'] = launch_date
k_var['state_changed_at'] = change_date
k_var['days_to_change'] = (k_var['state_changed_at'] - k_var['launched_at'])

In [ ]:
print k_var.dtypes

# Converting goal amount to USD currency units:

In [ ]:
k_var['goal_USD'] = k_var['goal'] * k_var['static_usd_rate']

# Converting catgory url to name

In [ ]:
import re
category = ''
for item in k_var['category']:
    m = re.search('(?<=/categories/)\w+', item)
    #print m.group(0) 
    category += m.group(0) + ','
# print category # containes multiple categories 
category_name = pd.DataFrame(category.split(','), columns = ['category'])

k_var['category_name'] = category_name
#k_var.head()

In [ ]:
print k_var.category_name.unique()
print k_var.groupby('category_name').size()

In [ ]:
# want to condense categories into bigger categories so that they have more of an effect
# when used in the regression.

# categories:
#    art + crafts : 16254 + 2979
#    design + fashion : 6733 + 8340
#    journalism + publishing + comics : 2242 + 19305 + 3525
#    food : 9765
#    technology : 18007
#    games : 11966
#    theater + dance + music : 2880 + 1051 + 28130
#    film + photography :  26512 + 19305
    

In [ ]:
#w['female'] = w['female'].map({'female': 1, 'male': 0})

#test1['category_name'] = test1['category_name'].map({'art': 'art+craft', 'crafts':'art+craft',
#                                                  'design': 'des + fash', 'fashion': 'des + fash'})

In [ ]:
k_var.category_name.unique()

In [ ]:
k_var.category_name.replace(['art', 'crafts', 'design', 'fashion','journalism', \
                            'publishing', 'comics', 'food', 'tech', 'games', \
                            'theater', 'dance', 'music', 'film', 'photography'], \
                           
                           ['art+craft', 'art+craft','des+fash','des+fash',\
                            'jour+pub+com', 'jour+pub+com', 'jour+pub+com', \
                            'food', 'tech', 'games','theat+dan+mus', \
                            'theat+dan+mus', 'theat+dan+mus',\
                            'film+photo', 'film+photo'
                           ], inplace=True)

In [ ]:
print k_var.category_name.unique()
print k_var.groupby('category_name').size()

# Converting blurb to count of words in blurb

In [ ]:
blurb_count = []
for blurb in k_var['blurb']:
    #print blurb
    try:
        blurb_count.append(len(blurb.split(' ')))
    except:
        blurb_count.append(blurb)
k_var['blurb_count'] = blurb_count
k_var.head()

# Removing states that are live

In [ ]:
k_var_state = k_var[k_var['state'] != 'live']
print len(k_var)
print len(k_var_state)
k_var_state.groupby(['state']).size()

# Cleaned kickstarter data:

In [ ]:
#kick_data = k_var_state.drop(['goal', 'static_usd_rate','currency','category'], axis=1)
kick_data = k_var_state[['name','category_name','blurb','blurb_count','goal_USD','backers_count','launched_at','state_changed_at','days_to_change','state']]
kick_data.head()

# Get going by asking the following questions and looking for the answers with some code and plots:

    Can you count something interesting?

    Can you find some trends (high, low, increase, decrease, anomalies)?

    Can you make a bar plot or a histogram?

    Can you compare two related quantities?

    Can you make a scatterplot?

    Can you make a time-series plot?


In [ ]:
# Can you count something interesting? 
# Looking at number of successful, canceled, live and suspended projects and 
# relative numb of backers per year per state.

g = kick_data[['backers_count','state']].groupby(['state', kick_data['launched_at'].dt.year,]).agg(['size','sum'])
g.unstack('state').fillna(0)

# Will have to divide sum (count of backers) by size (total number of projects) per year, per state 
# to get relative fraction of backers at that point.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# Can you find some trends (high, low, increase, decrease, anomalies)?
# Looking at number of projects launched per year and number of backers per year across all 
# states.

g_size = kick_data[['backers_count']].groupby([kick_data['launched_at'].dt.year,]).agg(['size'])
g_sum = kick_data[['backers_count']].groupby([kick_data['launched_at'].dt.year,]).agg(['sum'])

fig, axes = plt.subplots(nrows=2, ncols=1)
g_size.plot(ax=axes[0], kind= 'bar'); axes[0].set_title('total projects'); #plt.legend(loc='best')
g_sum.plot(ax=axes[1], kind = 'bar'); axes[1].set_title('total backers'); #plt.legend(loc='best')


# Conclusion: 1. There is not an equal number of projects or backers across years.
#             2. Number of backers and number of projects logicaly seem to correlate
#             3. As there are different number of projects launched per year, it is 
#                plausable that year has an effect on state of project and/or number of backers.
#             SOLUTION: Choose projects from recent years (2014-2016)

# Can you make a time-series plot?

In [ ]:
# Can you make a bar plot or a histogram?
# number of project per state over all years: will use k_var bc kick_data does not have live data in it.
# Conclustion: most projects, regardless of year, end up failing. 

%matplotlib inline
from matplotlib import pyplot as plt
from itertools import cycle, islice

my_colors = list(islice(cycle(['b', 'r', 'g', 'y','k']), None, len(k_var)))
g = k_var.groupby(['state']).size()
print g
g.plot(kind='bar', stacked = True, color=my_colors)

In [ ]:
# number of projects per state, per year: (kickstarter was launched, on April 28, 2009)

# want to get percentage of backers per state per year...
# (% will account for variation in # of projects per year, per state)

from matplotlib import pyplot as plt
from itertools import cycle, islice

my_colors = list(islice(cycle(['b', 'r', 'g']), None, len(k_var)))
g = k_var[k_var.state != 'live'][k_var.state != 'suspended'].groupby([k_var['launched_at'].dt.year, 'state']).size()
g = g.unstack().fillna(0)
print g #raw counts
g.plot(kind='bar', color=my_colors)

In [ ]:
# seems like 2016 - 2014 is current
# what may be driving this 2x increase in kickstarter

In [ ]:
# Number of backers per year per state 

kick_data_state = kick_data[kick_data.state != 'suspended']#[kick_data_state['launched_at'].dt.year != 2016]
g = kick_data_state[['backers_count','state','launched_at']].groupby(['state', kick_data_state['launched_at'].dt.year]).sum()
g = g.unstack('state')
print g
g.plot(kind='bar')
g.plot(kind='area')

#kick_data_state.dtypes

In [ ]:
kick_data_state.groupby([kick_data_state['launched_at'].dt.year]).count()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Can you compare two related quantities? 
    # Does number of backers correlate with number of words in blurb? linear regression
    # Does number of backers correlate with state? logistic regression

# Can you make a scatterplot?
 
fig = plt.figure(figsize=(5, 4))
fig, ax = plt.subplots()

x = kick_data[['blurb_count']]
y = kick_data[['backers_count']]
ax.set_title("blurb vs. backers")
ax.set_xlabel("blurb count")
ax.set_ylabel("backer count")

plt.plot(x, y, 'o', color='red')

                            ####Look for outliers in blurb count and backers count

In [ ]:
kick_corr = pd.DataFrame({'blurb count': kick_data['blurb_count'], 
                        'backer count': kick_data['backers_count']})
kick_corr.corr(method='pearson')

In [ ]:
#kick_data_state

In [ ]:
# Removing supspended and limiting dataset to recent years
kick_data_state = kick_data_state[kick_data_state.state != 'canceled'][kick_data_state['launched_at'].dt.year >= 2014]

In [ ]:
g = kick_data_state[['backers_count','state']].groupby(['state']).count()
my_colors = list(islice(cycle(['b', 'g', 'r']), None, len(kick_data_state[['state']])))
g.plot(kind = 'bar', color = my_colors)

# Logistic Regression 1

In [ ]:
import scipy.stats as stats
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

from sklearn import linear_model
from sklearn import preprocessing

# Initialize label encoder
label_encoder = preprocessing.LabelEncoder()

# Convert State variable to numeric
encoded_state = label_encoder.fit_transform(kick_data_state["state"])
print encoded_state
print list(label_encoder.classes_)
print list(label_encoder.inverse_transform([0,1]))

# Initialize logistic regression model
log_model = linear_model.LogisticRegression()

# Train the model
log_model.fit(y = pd.DataFrame(encoded_state), 
              X = pd.DataFrame(kick_data_state['backers_count']))

# Check trained model intercept
print log_model.intercept_

# Check trained model coefficients
print log_model.coef_

kick_corr2 = pd.DataFrame({'state': encoded_state, 
                        'backer count': kick_data_state['backers_count']})
kick_corr2.corr(method='pearson')

In [ ]:
print list(label_encoder.classes_)
print list(label_encoder.inverse_transform([0,1]))

In [ ]:
print kick_data_state.columns
print" ____________________________________________"
print kick_data_state.category_name.unique()

# Logistic Regression 2

In [ ]:
# Logistic Regression: backers_count and state
# http://patsy.readthedocs.io/en/latest/categorical-coding.html

from patsy import dmatrices

dta = kick_data_state

encoded_state = label_encoder.fit_transform(kick_data_state["state"])
kick_data_state['encoded_state'] = encoded_state

y, X = dmatrices('encoded_state ~ 0 + C(category_name) + blurb_count + backers_count + goal_USD', kick_data_state, return_type="dataframe")
print X.columns # have to -1 to get rid of intercept + so that art is not excluded

#kick_data_state
#print kick_data_state.columns
#kick_data_state[(kick_data_state.encoded_state == 2)]

In [ ]:
# column names for the dummy variables are ugly, so will rename those.

# fix column names of X
X = X.rename(columns = {'C(category_name)[art+craft]':'art+craft',
                        'C(category_name)[des+fash]':'des+fash',
                        'C(category_name)[film+photo]':'film+photo',
                        'C(category_name)[food]':'food',
                        'C(category_name)[games]':'games',
                        'C(category_name)[jour+pub+com]':'jour+pub+com',
                        'C(category_name)[technology]':'technology',
                        'C(category_name)[theat+dan+mus]':'theat+dan+mus'
                       })
print X.columns

In [ ]:
# flatten y into a 1-D array
y = np.ravel(y)
y

In [ ]:
from sklearn.linear_model import LogisticRegression

# instantiate a logistic regression model, and fit with X and y
model = LogisticRegression()
model = model.fit(X, y)

# check the accuracy on the training set
model.score(X, y) 

In [ ]:
# what percentage are not state failed [0]? 40% are successful 
y.mean()

In [ ]:
# examine the coefficients
coefficients = pd.DataFrame(zip(X.columns, np.transpose(model.coef_)))
coefficients

In [ ]:
# The higher the blurb_count and goal_USD the less likely for success
# The higher the backer count, the more likely for success

# Model Evaluation Using a Validation Set

In [ ]:
# So far, we have trained and tested on the same set. Let's instead split the data into a training set and a testing set.

from sklearn.cross_validation import train_test_split
# evaluate the model by splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
model2 = LogisticRegression()
model2.fit(X_train, y_train)

# check the accuracy on the training set
model.score(X_train, y_train) 

In [ ]:
predicted = model2.predict(X_test)
print predicted

In [ ]:
# generate class probabilities
probs = model2.predict_proba(X_test)
print probs

In [ ]:
from sklearn import metrics
# generate evaluation metrics
print metrics.accuracy_score(y_test, predicted)
#print metrics.roc_auc_score(y_test, probs[:, 1])

In [ ]:
print metrics.confusion_matrix(y_test, predicted)
print metrics.classification_report(y_test, predicted)

# Model Evaluation Using Cross-validation Set

In [ ]:
# evaluating the model using 10-fold cross-validation
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=10)
print scores
print scores.mean()

# Predicting the Probability

In [ ]:
# predict the probability of an art project with 50 blurb count, 50 backers and a goal of 500 dollars.

print list(label_encoder.inverse_transform([0,1]))
model.predict_proba(np.array([0,0,1,0,0,0,0,0,10,200,500])) 

In [ ]:
model.predict_proba(np.array([0,0,0,0,0,0,0,0,50,50,500]))

# Random Forest Regression

In [ ]:
                            #### How do we get categorical info into RFR?

In [ ]:
# making a dataframe with all features (called predictor_df)

#predictor_ df = X
predictor_df = kick_data_state[['backers_count', 'blurb_count', 'goal_USD']]
predictors = list(predictor_df.columns.values)
predictors

In [ ]:
# specify the number of features
numfeat = len(predictors)
numfeat

In [ ]:
# split the data into training/test sets...
dataset = kick_data_state
# test set:
testing = dataset[0:len(dataset):5]

In [ ]:
# here my y-value is a categorical variable (state)
Y_test = testing.state  
X_test = testing[predictors]

# training set: everything not the test set is the training set
#kick_data_state.index
keep = testing.index # unique filter for specific row
training = dataset[~dataset.index.isin(keep)]
Y_train = training.state
X_train = training[predictors]

In [ ]:
#Now use the optimal model's parameters to run random forest
from sklearn.ensemble import RandomForestClassifier

crf = RandomForestClassifier(n_estimators=1000, max_features=numfeat, max_depth=5) 
print("Parameters used in chosen RF model:\n " , crf.get_params())

In [ ]:
# Running Random Forest Regression

crf.fit(X_train, Y_train)

train_score = crf.score(X_train, Y_train)

print('crf training score is: %f' %(train_score))

test_score = crf.score(X_test, Y_test)

print('crf test score is: %f' %(test_score))

# Test, Train, Validation sets

In [ ]:
# Version two
# Use cross_validation.train_test_split, 2 times.

y, x = dmatrices('encoded_state ~ backers_count', kick_data_state, return_type="dataframe")

# First with (70,30) => (training, validation_test)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training, 30% test

# Secondly use (50,50) -> (validation, test).
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5) # 15% test, 15% validation

In [ ]:
model2 = LogisticRegression()
model2.fit(x_train, y_train)

In [ ]:
predicted = model2.predict(x_test)
print predicted

In [ ]:
# generate class probabilities
probs = model2.predict_proba(x_test)
print probs

In [ ]:
from sklearn import metrics
# generate evaluation metrics
print metrics.accuracy_score(y_test, predicted)
#print metrics.roc_auc_score(y_test, probs[:, 1])

In [ ]:
print metrics.confusion_matrix(y_test, predicted)
print metrics.classification_report(y_test, predicted)

# Are there repeates in data from sub-directories?? ...Yes!

In [ ]:
# What is kick_data_state? ...
# kick_data_state = kick_data_state[kick_data_state.state != 'canceled'][kick_data_state['launched_at'].dt.year >= 2014]

kick_data_state.columns

In [ ]:
#g = kick_data_state.groupby(['blurb'])
#size = g.size()
#size[size > 1].head()

In [ ]:
# people can resubmit their projects a 2nd, 3rd etc. time... i.e. there are repeates.
# Therefore, for this dataset we should only analyze the 1st submission (launched at the most recent date).

# This table has projects that have been submitted multiple times:
g_kick_data = kick_data_state.groupby(['blurb'])
g_filtered = g_kick_data.filter(lambda x: len(x) > 1).sort_values(['blurb','launched_at'])
len(g_filtered)

In [ ]:
g_filtered.head()

In [ ]:
for row in g_filtered.iterrows():
    print row

In [ ]:
len(kick_data_state)

In [ ]:
# This table has projects that have only been submitted once:
g2_filtered = g_kick_data.filter(lambda x: len(x) <= 1).sort_values(['blurb','launched_at'])
len(g2_filtered)

# 95978 + 1012 = 96990

# Testing accuracy of "live" data predictions

In [ ]:
# Combine 07-15 live rows into one csv. 
# Join '07-15 live' table w/ most resent data table (09/15 data).
# Find overlap of live and new rows based on same launched_at date and same blurb description
# select rows that have switched from live to failed/successful
# add identified rows to new csv file
# Predict state change on this dataset and compare to actual results to get accuracy

In [ ]:
pwd

In [ ]:
# What is k_var? Entire cleaned dataset with all outcomes

live_df = k_var[k_var.state == 'live']
live_df.to_csv('/Users/auroraleport/Documents/LePort_git/07_15_live.csv')

In [ ]:
kd915 = pd.read_csv('/Users/auroraleport/Documents/LePort_git/9_15_full')

In [ ]:
# This table has projects that have only been submitted once and are successes or failures:
g_kd915 = kd915[(kd915.state != 'live') & (kd915.state != 'suspended') & (kd915.state != 'canceled')].groupby(['blurb'])
kd915_filtered = g_kd915.filter(lambda x: len(x) <= 1).sort_values(['blurb','launched_at'])

In [ ]:
kd915_filtered.head()

In [ ]:
# are there live datasets that have multiple submissions?
g_live_df = live_df.groupby(['blurb'])
live_df_filtered = g_live_df.filter(lambda x: len(x) <= 1).sort_values(['blurb','launched_at'])

In [ ]:
# b4 taking out live from kd915
# print len(live_df_filtered) # there are no multiple submissions for live datasets
# print len(live_df)
# print len(kd915_filtered)
# print len(kd915)

In [ ]:
# after taking out live from kd915
print len(live_df_filtered) # there are no multiple submissions for live datasets
print len(live_df)
print len(kd915_filtered)
print len(kd915)

In [ ]:
# Combining tables to look for overlaps
live_kd915_filt_df = pd.concat([live_df, kd915_filtered], axis = 0)

In [ ]:
g_live_kd915_filt_df = live_kd915_filt_df.groupby(['blurb'])
live_woutcome = g_live_kd915_filt_df.filter(lambda x: len(x) > 1).sort_values(['blurb','launched_at'])
live_woutcome[['name','category_name','blurb','blurb_count','goal_USD','backers_count','launched_at',\
               'state_changed_at','days_to_change','state']]

# Unfiltered 09 data

In [ ]:
# kd915 is the full dataset, containing projects that have been submitted multiple times and 
# have live, successful, failed, suspended projects.
# Combining tables to look for overlaps

live_kd915_df = pd.concat([live_df, kd915], axis = 0)

In [ ]:
g_live_kd915_df = live_kd915_df.groupby(['blurb'])
live_woutcome_unfilt = g_live_kd915_df.filter(lambda x: len(x) > 1).sort_values(['blurb','launched_at'])
live_woutcome_unfilt[['name','category_name','blurb','blurb_count','goal_USD','backers_count','launched_at',\
               'state_changed_at','days_to_change','state']]

In [ ]:
# want to run live rows that have an outcome in 9_15 dataset through the algorithem to see if it predicts the
# correct outcome
# filtering for lives rows:

liveonly_live_woc = live_woutcome[live_woutcome.state == 'live']

In [ ]:
print len(liveonly_live_woc)

In [ ]:
# filtering for live projects that have been submitted multiple times.

notlive_live_woc = live_woutcome[(live_woutcome['state'] == 'failed') | (live_woutcome['state'] == 'successful')]
notlive_live_woc = notlive_live_woc[['name','category_name','blurb','blurb_count','goal_USD','backers_count',\
                               'launched_at','state_changed_at','days_to_change','state']]
print len(notlive_live_woc)
notlive_live_woc.head()

In [ ]:
liveonly_live_woc = liveonly_live_woc[['name','category_name','blurb','blurb_count','goal_USD','backers_count',\
                               'launched_at','state_changed_at','days_to_change','state']]

In [ ]:
liveonly_live_woc.head()

In [ ]:
# Converting category_name in live dataset to numerical values.
from patsy import dmatrices

encoded_state2 = label_encoder.fit_transform(liveonly_live_woc['state'])
liveonly_live_woc['encoded_state'] = encoded_state2

y_live, X_live = dmatrices('encoded_state2 ~ 0 + C(category_name) + blurb_count + backers_count + goal_USD', \
                           liveonly_live_woc, return_type='dataframe')

print X_live.columns 

In [ ]:
# column names for the dummy variables are ugly, so will rename those.

# fixing column names: 
X_live = X_live.rename(columns = {'C(category_name)[art+craft]':'art+craft',
                        'C(category_name)[des+fash]':'des+fash',
                        'C(category_name)[film+photo]':'film+photo',
                        'C(category_name)[food]':'food',
                        'C(category_name)[games]':'games',
                        'C(category_name)[jour+pub+com]':'jour+pub+com',
                        'C(category_name)[technology]':'technology',
                        'C(category_name)[theat+dan+mus]':'theat+dan+mus',
                       })
print X_live.columns


In [ ]:
X_live.head()

In [ ]:
predicted_live = model.predict(X_live)
predicted_live

In [ ]:
probs = model.predict_proba(X_live)
probs

In [ ]:
# Want to get a list of the live outcomes so that we can get accuracy of prediction
from sklearn import metrics

# generate evaluation metrics
y_outcomes = notlive_live_woc
y_outcomes_le = label_encoder.fit_transform(y_outcomes['state'])
print y_outcomes_le
print list(label_encoder.classes_)
print list(label_encoder.inverse_transform([0,1]))
print metrics.accuracy_score(y_outcomes_le, predicted_live)

In [ ]:
#print list(label_encoder.inverse_transform([0,1]))
# blurb_count = 21, backer_count = 0, goal_USD = 1100
model.predict_proba(np.array([0,0,1,0,0,0,0,0,30,50,1000]))

In [ ]:
coefficients

In [ ]:
k_var.describe()

In [ ]:
# based on trajectory of first week can we predict final backer count

In [ ]:
#EXTRA

In [ ]:
#7_15_live = k_var[k_var.state == 'live']

In [ ]:
#k_var.to_csv('9_15_full', index = False)